In [164]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import *
import random
import json
from datetime import datetime

In [165]:
nba_players = players.get_players()
nba_players = [player for player in nba_players if player['is_active'] == True]
nba_teams = teams.get_teams()

In [166]:
player = random.choice(nba_players)
player_id = player['id']

In [ ]:
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
df = player_info.common_player_info.get_data_frame()

In [ ]:
print(df['POSITION'])
print(df['DISPLAY_FIRST_LAST'])

In [ ]:
df.columns

In [ ]:
json_obj = json.loads(df.to_json(orient='records'))
json_obj[0]['BIRTHDATE']

In [ ]:
career = playercareerstats.PlayerCareerStats(player_id=player_id)
career = career.get_data_frames()[0]
career

In [ ]:
nba_teams = teams.get_teams()
nba_teams

In [ ]:
team = random.choice(nba_teams)
team_id = team['id']
team_info = teaminfocommon.TeamInfoCommon(team_id=team_id)
team_info

In [ ]:
team_info_df = team_info.team_info_common.get_data_frame()
team_info_df

In [ ]:
team_info_df.columns

In [ ]:
details = teamdetails.TeamDetails(team_id=team_id)
details_df = details.get_data_frames()[0]

In [ ]:
details_df

In [ ]:
player = random.choice(nba_players)
player_id = player['id']
player_profile = playerprofilev2.PlayerProfileV2(player_id=player_id)
df = player_profile.get_data_frames()[0]
json_obj = df.to_json(orient="records")
json_obj

In [ ]:
json_list = json.loads(json_obj)

In [ ]:
curr = json_list[len(json_list)-1]
curr

In [ ]:
df.columns

In [ ]:
sql_file_path = "../src/main/resources/data.sql"

team_sql_statements = []
player_sql_statements = []
for team in nba_teams:
    team_id = team['id']
    team_info = teaminfocommon.TeamInfoCommon(team_id)
    team_details = teamdetails.TeamDetails(team_id)
    team_info_obj = json.loads(team_info.team_info_common.get_data_frame().to_json(orient='records'))[0]
    team_details = team_details.get_data_frames()[0]
    coach = team_details['HEADCOACH'][0]
    owner = team_details['OWNER'][0]
    arena = team_details['ARENA'][0]
    team_sql_statements.append(f"INSERT INTO TEAM (id, name, city, coach, arena, founded, owner, conference) VALUES ({team['id']}, '{team['full_name']}', '{team['city']}', '{coach}', '{arena}', '{team['year_founded']}', '{owner}', '{team_info_obj['TEAM_CONFERENCE']}');")
    

for player in nba_players:
    try:
        player_id = player['id']
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        df = player_info.common_player_info.get_data_frame()
        json_list = json.loads(df.to_json(orient="records"))
        json_obj = json_list[len(json_list)-1]
        birthdate = datetime.strptime(json_obj['BIRTHDATE'], '%Y-%m-%dT%H:%M:%S')
        age = (datetime.now() - birthdate).days // 365
        player_sql_statements.append(f"INSERT INTO PLAYER (id, team_id, name, age, height, position) VALUES ({player_id}, {json_obj['TEAM_ID']}, '{json_obj['DISPLAY_FIRST_LAST']}', {age}, '{json_obj['HEIGHT']}', '{json_obj['POSITION'].upper()}');")
    except Exception as e:
        print(str(e))

with open(sql_file_path, 'a') as file:
    file.write("\n".join(team_sql_statements))
    file.write("\n")
    file.write("\n".join(player_sql_statements))

In [ ]:
with open(sql_file_path, 'r') as file:
    lines = file.readlines()

# Process each line to ensure it ends with a semicolon
with open(sql_file_path, 'w') as file:
    for line in lines:
        # Strip any existing whitespace or newline characters
        stripped_line = line.strip()
        
        # Append a semicolon if the line is not empty and doesn't already end with one
        if stripped_line and not stripped_line.endswith(';'):
            stripped_line += ';'
        
        # Write the modified line back to the file with a newline
        file.write(stripped_line + '\n')

In [ ]:
id = random.choice(nba_teams)['id']

In [ ]:
teamdashboard = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(team_id=team_id)

In [ ]:
teamdashboard = teamdashboard.get_data_frames()

In [ ]:
teamdashboard[1]

In [163]:
teamstats_statements = []
for team in nba_teams:
    id = team['id']
    json_list = json.loads(teamyearbyyearstats.TeamYearByYearStats(team_id=id).get_data_frames()[0].to_json(orient='records'))
    for obj in json_list:
        gp = obj['GP']
        teamstats_statements.append(f"INSERT INTO TEAMSTATS (team_id, season, played, won, lost, ppg, rpg, apg, fgpercent, ftpercent, fgthreepercent, conf_rank) VALUES ({id}, '{obj['YEAR']}', {gp}, {obj['WINS']}, {obj['LOSSES']}, {obj['PTS']/gp}, {obj['REB']/gp}, {obj['AST']/gp}, {obj['FG_PCT']}, {obj['FT_PCT']}, {obj['FG3_PCT']}, {obj['CONF_RANK']});")

with open(sql_file_path, 'a') as file:
    file.write("\n".join(teamstats_statements))

In [ ]:
team_by_year = teamyearbyyearstats.TeamYearByYearStats(team_id=id)

In [ ]:
teamstats = team_by_year.get_data_frames()

In [ ]:
df = teamstats[0]
df

In [158]:
df.columns

Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'CONF_COUNT', 'DIV_COUNT', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'PF', 'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK'],
      dtype='object')

In [ ]:
standings = leaguestandings.LeagueStandings()

In [ ]:
standings.get_data_frames()

In [ ]:
df.columns

In [169]:
player = random.choice(nba_players)
player_id = player['id']

In [175]:
sql_file_path = "../src/main/resources/data.sql"
playerstats_statements = []
for player in nba_players:
    player_id = player['id']
    career = playercareerstats.PlayerCareerStats(player_id=player_id)
    stats_list = json.loads(career.get_data_frames()[0].to_json(orient='records'))
    for season in stats_list:
        gp = season['GP']
        playerstats_statements.append(f"INSERT INTO PLAYERSTATS (player_id, season, minpergame, ppg, rpg, apg, stealspergame, bpg, tov, fgpercent, ftpercent, fgthreepercent) VALUES ({player_id}, '{season['SEASON_ID']}', {season['MIN']/gp}, {season['PTS']/gp}, {season['REB']/gp}, {season['AST']/gp}, {season['STL']/gp}, {season['BLK']/gp}, {season['TOV']/gp}, {season['FG_PCT']}, {season['FT_PCT']}, {season['FG3_PCT']})")

with open(sql_file_path, 'a') as file:
    file.write('\n'.join(playerstats_statements))

In [168]:
career.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [174]:
career = playercareerstats.PlayerCareerStats(player_id=player_id)
stats_list = json.loads(career.get_data_frames()[0].to_json(orient='records'))
for season in stats_list:
    print(season)

{'PLAYER_ID': 1641764, 'SEASON_ID': '2023-24', 'LEAGUE_ID': '00', 'TEAM_ID': 1610612744, 'TEAM_ABBREVIATION': 'GSW', 'PLAYER_AGE': 21.0, 'GP': 74, 'GS': 28, 'MIN': 1968.0, 'FGM': 271, 'FGA': 597, 'FG_PCT': 0.454, 'FG3M': 90, 'FG3A': 234, 'FG3_PCT': 0.385, 'FTM': 50, 'FTA': 79, 'FT_PCT': 0.633, 'OREB': 118, 'DREB': 309, 'REB': 427, 'AST': 273, 'STL': 61, 'BLK': 14, 'TOV': 88, 'PF': 122, 'PTS': 682}
